##### 아래 URL의 NBA 데이터를 크롤링하여 판다스 데이터 프레임으로 나타내세요. 
- http://stats.nba.com/teams/traditional/?sort=GP&dir=-1

##### 셀레니움을 이용하여 네이버 IT/과학 기사의 10 페이지 까지의 최신 제목 리스트를 크롤링하세요.
- http://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=105